In [ ]:
HTML(read(open("style.css"), String))

# Iterative Deepening

This notebook implements the iterative deepening algorithm. The iterative deepening algorithm in an improvement of the alpha-beta-pruning algorithm and will increase the performance of the search. 

The iterative deepening algorithm iterates over the depth starting at a depth of 1. It will search the game tree to a depth of 1. Using the evaluation it can then order the next moves after their evaluation. Moves with a good evaluation have a high chance of being actual good moves, resulting in more paths pruned when searching the game tree. It will run until it has reached the maximum depth. When it finds a forced mate at any stage, this is also the shortest way of mating the opponent.

Additionally iterative deepening allows to increase the maximum depth easily. Therefore the algorithm can keep searching even if the maximum depth has been reached if the search has only been a very short time. This is important in the endgame of chess, as the number of possible moves in the endgame is not as as the middlegame but therefore requires to think more into the depth.

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("DataStructures")
using DataStructures
# Pkg.add("NBInclude")
using NBInclude
using BenchmarkTools

In [ ]:
@nbinclude("AdvancedBoard.ipynb")

In [ ]:
@nbinclude("Memoization.ipynb")

In [ ]:
@nbinclude("QuiescenceSearch.ipynb")

## maxValue

The function `maxValue` returns the minimal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. All possible moves are sorted by a `PriorityQueue` using the evaluation of the position. Good moves will be prioritized which will increase the chance of pruning paths. 

Arguments:
1. `aBoard::AdvBoard` is a chess `state`
1. `depth::Int64` is the number of halfmoves the engine should analyze before terminating 
1. `alpha::Int64` is a minimal value that has been calculated during the recursive process
1. `beta::Int64`  is a maximal value that has been calculated during the recursive process
1. `flagQuiesce::Bool` is an optional flag specifies whether the quiesce-search should be used. If not then the `see`-variant of the quiesce-search will be used.

In [ ]:
function maxValue(aBoard::AdvBoard, depth::Int64,
                  cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64, beta::Int64, flagQuiesce::Bool=false)::Int64
    if isterminal(aBoard.state) 
        return terminal_evaluation(aBoard)
    end
    if depth == 0
        # return aBoard.score
        if flagQuiesce
            return quiesceMax(aBoard, 0, alpha, beta)
            # return evaluate(aBoard, quiesceMax, 0, cache, alpha, beta)
        else
            return aBoard.score
        end
    end
    value = alpha
    queue = PriorityQueue{Move, Int64}()
    for move in moves(aBoard.state)
        nextHash = zobrist_hash(aBoard.state, aBoard.hash, move)
        val = value_cache(nextHash, depth-2, cache)
        if val == nothing
            val = -200000
        end
        enqueue!(queue, move, -val)
    end
    while !isempty(queue)
        move = peek(queue)[1]
        undo = domoveAdv!(aBoard, move)
        value = max(value, evaluate(aBoard, minValue,  depth - 1, cache, value, beta, flagQuiesce))
        undomoveAdv!(aBoard, undo)
        if value >= beta
            return value
        end
        delete!(queue, move)
    end
    return value
end

## minValue

The function `minValue` returns the maximal centipawn evaluation of the current position for the player playing black where both players have played the optimal moves according to the algorithm and terminating after the given depth. All possible moves are sorted by a `PriorityQueue` using the evaluation of the position. Good moves will be prioritized which will increase the chance of pruning paths. 

Arguments:
1. `aBoard::AdvBoard` is a chess `state`
1. `depth::Int64`     is the number of halfmoves the engine should analyze before terminating 
1. `alpha::Int64`     is a minimal value that has been calculated during the recursive process
1. `beta::Int64`      is a maximal value that has been calculated during the recursive process
1. `flagQuiesce::Bool`is an optional flag specifies whether the quiesce-search should be used. If not then the `see`-variant of the quiesce-search will be used.

In [ ]:
function minValue(aBoard::AdvBoard, depth::Int64,
                  cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64, beta::Int64, flagQuiesce::Bool=false)::Int64
    if isterminal(aBoard.state)
        return terminal_evaluation(aBoard)
    end
    if depth == 0
        # return aBoard.score
        if flagQuiesce
            return quiesceMin(aBoard,  0,  alpha, beta) 
            # return evaluate(aBoard,quiesceMin , 0, cache, alpha, beta) with Transposition
        else
            return aBoard.score
        end
    end
    value = beta
    queue = PriorityQueue{Move, Int64}()
    for move in moves(aBoard.state)
        nextHash = zobrist_hash(aBoard.state, aBoard.hash, move)
        val = value_cache(nextHash, depth-2, cache)
        if val == nothing
            val = 200000
        end
        enqueue!(queue, move, val)
    end
    while !isempty(queue)
        move = peek(queue)[1]
        Undo = domoveAdv!(aBoard, move)
        value = min(value, evaluate(aBoard, maxValue, depth - 1, cache, alpha, value, flagQuiesce))
        undomoveAdv!(aBoard, Undo)
        if value <= alpha
            return value
        end
        delete!(queue, move)
    end
    return value
end

The function `value_cache` is a helping function for the `minValue` and `maxValue` function. It takes in 2 arguments:
1. `hash::UInt64` is a chess `state` of type `Board`
1. `depth::Int64` is the number of halfmoves the engine should analyze before terminating
1. `cache::Dict{UInt64, Tuple{String, Int64, Int64}}` is the cache that is searched

The function looks into the cache and returns any previously saved values for this position. This information is used to sort good moves inside of the PriorityQueue.
The function returns the value for this `hash` if the `hash` is in the Cache and has a sufficient pre-calculated depth. If the cache does not have an entry for this `hash` or entry does not have sufficient depth the function will return `nothing`.

In [ ]:
function value_cache(hash::UInt64, depth::Int64, cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    tuple::Tuple{String, Int64, Int64} = get(cache, hash, ("", 0, 0))
    if tuple != ("", 0, 0)
        _, value, d = tuple
        if d >= depth
            return value
        end
    end
    # new move or no entry with enough depth
    return nothing
end

### Function: pd_evaluate
This function performs iterative deepening search on a given state of the board using the evaluation function f and caching the results using the dictionary cache.

Arguments:
1. `State::Board` is the current state of the board
1. `f::Function` is the evaluation function that will be called
1. `depth::Int64` is the maximum depth of the search
1. `cache::Dict{UInt64, Tuple{String, Int64, Int64}}` is the dictionary to cache the results

Returns:
1. `bestVal`: The best value found in the search.
1. `depth`: The depth at which the best value was found.

In [ ]:
function pd_evaluate(aBoard::AdvBoard, f::Function, depth::Int64, 
                     cache::Dict{UInt64, Tuple{String, Int64, Int64}}, increaseDepth::Bool= false,showTimes::Bool = false)
    bestVal = aBoard.score
    # println("Boards score ", score)
    # start time
    starttime = time()
    d = 1
    alpha = -100000
    beta = 100000
    while d <= depth
        flagQuiesce = d == depth ? true : false
        bestVal = evaluate(aBoard, f, d, cache, alpha, beta, flagQuiesce)
        if abs(bestVal) == 100000
            return bestVal, d
        end
        # Check if is d equals depth and the difference between startime and currenttime increases depth
        if showTimes
            println("Depth is ", d, " and time needed is ", time() - starttime)
        end
        if increaseDepth && d == depth && time() - starttime < 5
            depth += 1 
        end
        d +=1
    end
    return bestVal, depth
end

### Function: iterativeDeepening
This function performs iterative deepening search on the given State and returns the best move found.

Arguments:
1. `aBoard::AdvBoard` is a chess board in the current state
1. `depth::Int64` is the maximum depth to search
1. `cache::Dict{UInt64, Tuple{String, Int64, Int64}}` is the cache dictionary to store the evaluated board states. It is optional and initializes an empty Dictionary by default.

Returns:
1. `bestVal`: The best score found by the iterative deepening search.
1. `BestMove`: The best move found by the iterative deepening search.

In [ ]:
function iterativeDeepening(aBoard::AdvBoard,  depth::Int64, 
                            cache::Dict{UInt64, Tuple{String, Int64, Int64}} = initCache())
    side = sidetomove(aBoard.state)
    bestVal, depth = (side == WHITE) ? pd_evaluate(aBoard, maxValue, depth, cache, true) : 
                                       pd_evaluate(aBoard, minValue, depth, cache, true)
    next_moves = moves(aBoard.state)
    
    BestMoves::Array{Move} = []

    queue = PriorityQueue{Move, Int64}()
    for move in next_moves
        nextHash = zobrist_hash(aBoard.state, aBoard.hash, move)
        val = value_cache(nextHash, depth-2, cache)
        if val == nothing
            val = 200000
        else
            if side == WHITE
                val = -1 * val
            end
        end
        enqueue!(queue, move, val)
    end
    
    while !isempty(queue)
        move = peek(queue)[1]
        undoinfo = domoveAdv!(aBoard, move)
        if side == WHITE
            if evaluate(aBoard, minValue, depth-1, cache, -100000, 100000, true) == bestVal
                append!(BestMoves, [move])
                undomoveAdv!(aBoard, undoinfo)
                break
            end
        else 
            if evaluate(aBoard, maxValue, depth-1, cache, -100000, 100000, true) == bestVal
                append!(BestMoves, [move])
                undomoveAdv!(aBoard, undoinfo)
                break
            end
        end
        undomoveAdv!(aBoard, undoinfo)
        delete!(queue, move)
    end
    
    # ------------------------------------------------------------------------------------------
    # Variation? Klappt nur, wenn einer der besten Pfade mit "=" flag im Cache schon gespeichert ist. 
    # Wirft Fehler, wenn man Ruhesuche verwendet
    # BestMoves::Array{Move} = [move for move in next_moves if zobrist_hash(aBoard.state, aBoard.hash, move) in keys(cache) &&
    #                                                       cache[zobrist_hash(aBoard.state, aBoard.hash, move)][1] == "=" &&
    #                                                       cache[zobrist_hash(aBoard.state, aBoard.hash, move)][2] == bestVal]
    # ------------------------------------------------------------------------------------------
    
    # Debug
    # for move in next_moves
    #   nextHash = zobrist_hash(aBoard.state, aBoard.hash, move)
    #   if nextHash in keys(cache)
    #       println(cache[nextHash])
    #   else
    #       println("Move ", move, " not in cache.")
    #   end
    # end
    BestMove::Move = rand(BestMoves)
    
    return bestVal, BestMove
end